In [1]:
"""
This file is part of DGPMP2-ND    

Copyright (C) 2024 ArtiMinds Robotics GmbH

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
"""

import os
from pathlib import Path
import open3d as o3d
from open3d.geometry import TriangleMesh
import torch

from diff_gpmp2.differentiable_robot_model.diff_robot_model import DiffRobotModel
from diff_gpmp2.utils.sdf import CollisionObject, SignedDistanceField

NOTEBOOK_DIR = Path(os.path.abspath(os.path.dirname("__file__")))


boxes = [
    CollisionObject(torch.tensor([-0.5, -0.1, 0.1, 0, 0, 0]), TriangleMesh.create_box(width=0.2, height=0.2, depth=0.2)),
    CollisionObject(torch.tensor([-0.6, -0.2, 0.2, 0, 0, 0]), TriangleMesh.create_box(width=0.1, height=0.1, depth=0.1)),
    CollisionObject(torch.tensor([-0.2, -0.5, 0.05, 0, 0, 0]), TriangleMesh.create_box(width=0.1, height=0.1, depth=0.1)),
    CollisionObject(torch.tensor([0, 0.5, 0.05, 0, 0, 0]), TriangleMesh.create_box(width=0.1, height=0.1, depth=0.1))
]
robot = DiffRobotModel(
    urdf_path=(NOTEBOOK_DIR / 'urdf' / 'ur5' / 'robot.urdf').as_posix(),
    good_initial_conf=torch.deg2rad(torch.tensor([23, -86, 94, -98, -90, 113]))
)
sdf = SignedDistanceField.from_obstacles(
    obstacles=boxes,
    collision_padding=0.01,
    voxel_size=0.01)

start_conf = torch.deg2rad(torch.tensor([-46, -79, 112, -123, -90, 43]))
start_vel = torch.zeros(6)
start = torch.cat((start_conf, start_vel))
goal_conf = torch.deg2rad(torch.tensor([26, -76, 108, -122, -90, 116]))
goal_vel = torch.zeros(6)
goal = torch.cat((goal_conf, goal_vel))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from diff_gpmp2.gpmp2.diff_gpmp2_planner import DiffGPMP2Planner
from diff_gpmp2.gpmp2.factors import Factors
from diff_gpmp2.gpmp2.gp.gp_factor import GPFactor
from diff_gpmp2.gpmp2.gp.prior_factor import PriorFactor
from diff_gpmp2.gpmp2.gp.robot_limits_factor import RobotLimitsFactor
from diff_gpmp2.gpmp2.obstacle.obstacle_factor import ObstacleFactor
from diff_gpmp2.gpmp2.plan_layer import OptimParams
from diff_gpmp2.utils.planner_utils import straight_line_traj

device = torch.device('cpu')

# Factors
batch_size = 1
state_dim = robot.n_dofs * 2  # position & velocity
wksp_dim = 3
total_time_sec = 10
total_time_step = 20

gp_prior = GPFactor(
    name="GP",
    weight=torch.tensor(1),
    dof=robot.n_dofs,
    device=device,
    threshold=10.0
)
gp_prior.set_total_time_sec(total_time_sec)
start_prior = PriorFactor(
    name="Start",
    meanb=start,
    K=torch.tensor(0.01),
    state_dim=state_dim,
    thb_point_idx=0,
    device=device,
    threshold=1e-3,
)
goal_prior = PriorFactor(
    name="Goal",
    meanb=goal,
    K=torch.tensor(0.01),
    state_dim=state_dim,
    thb_point_idx=-1,
    device=device,
    threshold=1e-3,
)
obs_factor = ObstacleFactor(
    name="Obst.",
    state_dim=state_dim,
    weight=torch.tensor(0.05),
    threshold=0,
    eps=0.001,
    sdf=sdf,
    robot_model=robot,
    device=device,
)
lims_factor = RobotLimitsFactor(
    name="Limits",
    state_dim=state_dim,
    robot_model=robot,
    K=torch.tensor(0.01),
    device=device,
    threshold=0
)

factors = Factors(
    state_dim=state_dim,
    batch_size=batch_size,
    device=device,
    factors=[start_prior, goal_prior, gp_prior, obs_factor, lims_factor],
)

optim_params = OptimParams(method="gauss_newton", reg=0.1, init_lr=1, max_iters=100,
                           error_patience=15)

planner = DiffGPMP2Planner(
    factors=factors,
    optim_params=optim_params,
    state_dim=state_dim,
    batch_size=batch_size,
    device=torch.device('cpu')
)

In [3]:
from tempfile import TemporaryDirectory
import cv2

th_init = straight_line_traj(
    start_conf, goal_conf, total_time_sec, total_time_step, robot.n_dofs, device
)

obs_factor.plot_trajectory(th_init.unsqueeze(0))

In [4]:
th_final, err_per_iter, k = planner(th_init, verbose=True)

{'Start': 5.960464477539063e-08, 'Goal': 1.1920928955078125e-07, 'GP': 0.12716001272201538, 'Obst.': 0.4467200040817261, 'Limits': 0.0}
{'Start': 3.516674041748047e-06, 'Goal': 2.9772520065307617e-05, 'GP': 1.1861517429351807, 'Obst.': 0.3933199942111969, 'Limits': 0.0}
{'Start': 6.020069122314453e-06, 'Goal': 3.978610038757324e-05, 'GP': 1.8765300512313843, 'Obst.': 0.39169999957084656, 'Limits': 0.0}
{'Start': 6.556510925292969e-06, 'Goal': 4.360079765319824e-05, 'GP': 1.6250630617141724, 'Obst.': 0.3894599974155426, 'Limits': 0.0}
{'Start': 9.775161743164062e-06, 'Goal': 4.690885543823242e-05, 'GP': 1.9955742359161377, 'Obst.': 0.3844600021839142, 'Limits': 0.0}
{'Start': 1.2278556823730469e-05, 'Goal': 4.926323890686035e-05, 'GP': 2.253143548965454, 'Obst.': 0.37386003136634827, 'Limits': 0.0}
{'Start': 9.417533874511719e-06, 'Goal': 5.1140785217285156e-05, 'GP': 2.1540846824645996, 'Obst.': 0.3797999918460846, 'Limits': 0.0}
{'Start': 9.894371032714844e-06, 'Goal': 5.3256750106811

In [5]:
obs_factor.plot_trajectory(th_final.unsqueeze(0))